<a href="https://colab.research.google.com/github/bgsDoneit/Azure-AI-Foundry-steup/blob/master/gcp_rag_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Overview

Retrieval Augmented Generation (RAG) improves Large Language Models (LLMs) by allowing them to access and process external information sources during generation. This ensures the model's responses are grounded in factual data and avoids hallucinations.

A common problem with LLMs is that they don't understand private knowledge, that
is, your organization's data. With RAG Engine, you can enrich the
LLM context with additional private information, because the model can reduce
hallucinations and answer questions more accurately.

By combining additional knowledge sources with the existing knowledge that LLMs
have, a better context is provided. The improved context along with the query
enhances the quality of the LLM's response.

The following concepts are key to understanding Vertex AI RAG Engine. These concepts are listed in the order of the
retrieval-augmented generation (RAG) process.

1. **Data ingestion**: Intake data from different data sources. For example,
  local files, Google Cloud Storage, and Google Drive.

1. **Data transformation**: Conversion of the data in preparation for indexing. For example, data is split into chunks.

1. **Embedding**: Numerical representations of words or pieces of text. These numbers capture the
   semantic meaning and context of the text. Similar or related words or text
   tend to have similar embeddings, which means they are closer together in the
   high-dimensional vector space.

1. **Data indexing**: RAG Engine creates an index called a corpus.
   The index structures the knowledge base so it's optimized for searching. For
   example, the index is like a detailed table of contents for a massive
   reference book.

1. **Retrieval**: When a user asks a question or provides a prompt, the retrieval
  component in RAG Engine searches through its knowledge
  base to find information that is relevant to the query.

1. **Generation**: The retrieved information becomes the context added to the
  original user query as a guide for the generative AI model to generate
  factually grounded and relevant responses.

For more information, refer to the public documentation for [Vertex AI RAG Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/rag-overview).

## Get started

### Install Vertex AI SDK and Google Gen AI SDK


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 8.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.


### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [2]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [11]:
# Use the environment variable if the user doesn't provide Project ID.
import os

from google import genai
import vertexai

PROJECT_ID = "mygcpaiagentlanggraphtavliy"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "europe-west3")

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### Import libraries

In [12]:
from IPython.display import Markdown, display
from google.genai.types import GenerateContentConfig, Retrieval, Tool, VertexRagStore
from vertexai import rag

### Create a RAG Corpus

In [13]:
# Currently supports Google first-party embedding models
EMBEDDING_MODEL = "publishers/google/models/text-embedding-005"  # @param {type:"string", isTemplate: true}

rag_corpus = rag.create_corpus(
    display_name="my-rag-corpus1",
    backend_config=rag.RagVectorDbConfig(
        rag_embedding_model_config=rag.RagEmbeddingModelConfig(
            vertex_prediction_endpoint=rag.VertexPredictionEndpoint(
                publisher_model=EMBEDDING_MODEL
            )
        )
    ),
)

### Check the corpus just created

In [17]:
rag.list_corpora()

ListRagCorporaPager<rag_corpora {
  name: "projects/mygcpaiagentlanggraphtavliy/locations/europe-west3/ragCorpora/2305843009213693952"
  display_name: "my-rag-corpus"
  create_time {
    seconds: 1765987969
    nanos: 354544000
  }
  update_time {
    seconds: 1765987969
    nanos: 354544000
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
      knn {
      }
    }
    rag_embedding_model_config {
      vertex_prediction_endpoint {
        endpoint: "projects/mygcpaiagentlanggraphtavliy/locations/europe-west3/publishers/google/models/text-embedding-005"
      }
    }
  }
}
rag_corpora {
  name: "projects/mygcpaiagentlanggraphtavliy/locations/europe-west3/ragCorpora/6917529027641081856"
  display_name: "my-rag-corpus1"
  create_time {
    seconds: 1765988285
    nanos: 8692000
  }
  update_time {
    seconds: 1765988285
    nanos: 8692000
  }
  corpus_status {
    state: ACTIVE
  }
  vector_db_config {
    rag_managed_db {
      knn {
      }
    }
 

### Upload a local file to the corpus

In [15]:
%%writefile test.md


Apollo Clinic
#1, Near Mc.Donalds, Prakasam Salai, Valasaravakkam, Chennai, Tamil Nadu, India - 600087
Phone No: 1860 500 7788
Email: kothandaram.rj@apolloclinic.com

Patient Name: RAMANI BHASKARAN
Age/Gender: 58Y/Female
Physician: Dr. Madhavi

Purpose of Visit: Routine Health Check

PHYSICAL EXAMINATION:
- Weight: 69 Kg
- Height: 157 cm
- BMI: 27.99
- Pulse: 85 BPM
- SpO2: 98%
- Temperature: 97°F
- BP: 150/80 mmHg
- Respiratory Rate: 16/min

OBSERVATIONS:
- Pallor: No
- Icterus: No
- Cyanosis: No
- Clubbing: No
- Koilonychia: No
- Edema: No

IMPRESSIONS:
- Pre Diabetic Stage
- Hypertension
- Vitamin D Deficiency
- Vitamin B12 Deficiency
- Cervical Spondylosis
- Bilateral Carotid Plaques

LAB RESULTS SUMMARY:

Blood:
- PCV: 35.90% (Low)
- HbA1c: 6.2%
- eAG: 131 mg/dL
- Hemoglobin: 12 g/dL
- RBC Count: 4.07 M/cu.mm
- TLC: 7,900
- Platelet Count: 249,000

Lipid Profile:
- Total Cholesterol: 170 mg/dL
- Triglycerides: 199 mg/dL (High)
- HDL: 43 mg/dL
- LDL: 87.2 mg/dL
- VLDL: 39.8 mg/dL (High)

Renal Function:
- Creatinine: 0.74 mg/dL
- Urea: 16.00 mg/dL (Low)
- BUN: 7.5 mg/dL (Low)

Liver Function:
- Bilirubin Total: 0.47 mg/dL
- ALT/SGPT: 15 U/L
- AST/SGOT: 15 U/L
- Albumin: 4.5 g/dL

Thyroid Profile:
- T3: 0.83 ng/mL
- T4: 7.43 µg/dL
- TSH: 3.931 µIU/mL

Vitamin Levels:
- Vitamin B12: 50 pg/mL (Low)
- Vitamin D (25-OH): 15.4 ng/mL (Low)

Urine Examination:
- pH: 5.5
- Specific Gravity: 1.025
- All parameters: Normal

Other Tests:
- Rheumatoid Factor: Negative
- CA-125: 6.3 U/mL

AI RISK SCORES:
- Risk of Heart Disease: 4 (Low Risk)
- Prediabetes: 2.5/100 (Low Risk)
- COPD Risk Assessment: 27% (Low Risk)

Note: Radiology reports are available separately.

DISCLAIMER: All lab results are subject to clinical interpretation by qualified medical professionals and not to be used for medicolegal purposes.



Writing test.md


In [16]:
rag_file = rag.upload_file(
    corpus_name=rag_corpus.name,
    path="test.md",
    display_name="test.md",
    description="my test file",
)

### Import files from Google Cloud Storage

Remember to grant "Viewer" access to the "Vertex RAG Data Service Agent" (with the format of `service-{project_number}@gcp-sa-vertex-rag.iam.gserviceaccount.com`) for your Google Cloud Storage bucket.

For this example, we'll use a public GCS bucket containing earning reports from Alphabet.

In [ ]:
# INPUT_GCS_BUCKET = (
#     "gs://cloud-samples-data/gen-app-builder/search/alphabet-investor-pdfs/"
# )

# response = rag.import_files(
#     corpus_name=rag_corpus.name,
#     paths=[INPUT_GCS_BUCKET],
#     # Optional
#     transformation_config=rag.TransformationConfig(
#         chunking_config=rag.ChunkingConfig(chunk_size=1024, chunk_overlap=100)
#     ),
#     max_embedding_requests_per_min=900,  # Optional
# )

### Import files from Google Drive

Eligible paths can be formatted as:

- `https://drive.google.com/drive/folders/{folder_id}`
- `https://drive.google.com/file/d/{file_id}`.

Remember to grant "Viewer" access to the "Vertex RAG Data Service Agent" (with the format of `service-{project_number}@gcp-sa-vertex-rag.iam.gserviceaccount.com`) for your Drive folder/files.


In [ ]:
# response = rag.import_files(
#     corpus_name=rag_corpus.name,
#     paths=["https://drive.google.com/drive/folders/{folder_id}"],
#     # Optional
#     transformation_config=rag.TransformationConfig(
#         chunking_config=rag.ChunkingConfig(chunk_size=512, chunk_overlap=50)
#     ),
# )

### Optional: Perform direct context retrieval

In [ ]:
# Direct context retrieval
response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus.name,
            # Optional: supply IDs from `rag.list_files()`.
            # rag_file_ids=["rag-file-1", "rag-file-2", ...],
        )
    ],
    rag_retrieval_config=rag.RagRetrievalConfig(
        top_k=10,  # Optional
        filter=rag.Filter(
            vector_distance_threshold=0.5,  # Optional
        ),
    ),
    text="Get me a summary of this health report?",
)
print(response)

# Optional: The retrieved context can be passed to any SDK or model generation API to generate final results.
# context = " ".join([context.text for context in response.contexts.contexts]).replace("\n", "")

### Create RAG Retrieval Tool

In [18]:
# Create a tool for the RAG Corpus
rag_retrieval_tool = Tool(
    retrieval=Retrieval(
        vertex_rag_store=VertexRagStore(
            rag_corpora=[rag_corpus.name],
            similarity_top_k=10,
            vector_distance_threshold=0.5,
        )
    )
)

### Generate Content with Gemini using RAG Retrieval Tool

In [ ]:
MODEL_ID = "gemini-2.5-pro"

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents="Whats my BMI looks like??",
    config=GenerateContentConfig(tools=[rag_retrieval_tool]),
)

display(Markdown(response.text))